# Importing Required Libraries

In [1]:
#All the requried libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib
import tkinter as tk
from tkinter import *
from tkinter.ttk import *
from tkinter import ttk
import time
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from pandas import DataFrame
import sklearn.metrics
import warnings
warnings.filterwarnings('ignore')

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
import matplotlib.image as mpimg

# User Variables

In [2]:
# CSV file path
CSV = "IrisData.csv"

# Use real dataset features names
realValues = 1 # Set to 0 to use features as {X1, X2, X3, X4}, Set to 1 to use features real names

# Color palette used [ DARKER, DARK, ACCENT ]
palette = ['#313131','#414141' ,'#FFFFFF' ] # Dark palette

# To add a custom accent, add Its name in the themeOptions list then its value in the themeColors list
themeOptions=[ "White", "Red", "Orange", "Cyan", "Green", "Blue", "Pink","Purple", "Yellow"]

themeColors=[ '#FFFFFF' #White
             ,'#CA3E47' #Red
             ,'#E16428' #Orange
             ,'#00ADB5' #Cyan
             ,'#4E9F3D' #Green
             ,'#0D63A5' #Blue
             ,'#B958A5' #Pink
             ,'#8685EF' #Purple
             ,'#FECD1A' #Yellow
            ]

# Dictionary used to carry theme colors
themeDict=dict()
for i in  range(len(themeOptions)):
    themeDict[themeOptions[i]] = themeColors[i]
    
# Size of charts
plotLength=8 # along X-axis
plotWidth=4 # along Y-axis

# Model constants
RATIO = 0.6 # Ratio of training data to available data

# Constants

In [3]:
# GUI Dropdown menu options
SetOptions = [ "Iris-setosa", "Iris-versicolor", "Iris-virginica" ]

# Available Features
if realValues:
    FeatureOptions=[ "Sepal Length", "Sepal Width", "Petal Length", "Petal Width" ]
else:
    FeatureOptions=[ "X1", "X2", "X3", "X4" ]

# Preprocessing the data

In [4]:
# Load the csv data
df = pd.read_csv(CSV)
df = df.rename(columns={'X1': FeatureOptions[0], 'X2': FeatureOptions[1],'X3': FeatureOptions[2],'X4': FeatureOptions[3]})

# Split the dataframe into a frame for each class
dfSetosa, dfVersicolor, dfVirginica = [x for _, x in df.groupby(df['Class'])]

#dfSetosa.head()
#dfVirginica.head()
#dfVersicolor.head()

# Assisting Functions

In [5]:
# Function that splits 2 dataframes into train and test frames
def train_testsplit(frame1, frame2, SEED):
    
    df1 = frame1
    df2 = frame2
    
    train1 = df1.sample(frac=RATIO,random_state=SEED) #random state is a seed value
    test1 = df1.drop(train1.index)
    
    train2 = df2.sample(frac=RATIO,random_state=SEED) #random state is a seed value
    test2 = df2.drop(train2.index)
    
    dftrain = pd.concat([train1, train2], ignore_index=True)
    dftest = pd.concat([test1, test2], ignore_index=True)
    
    return dftrain, dftest

# Function that splits a dataframe(train or test) into features and output frames
def X_ysplit(df, feature1, feature2):
    
    FEATURE1 = df.columns.get_loc(feature1)
    FEATURE2 = df.columns.get_loc(feature2)
    y = df.iloc[:,-1]
    X = pd.concat([df.iloc[:,FEATURE1], df.iloc[:,FEATURE2]], axis=1)
    
    return X,y

# Signum Function
def signum(y):    
    if y>=0:
        y=1
    else:
        y=-1
    return y

# Function that converts the output frame according to 2 features
def convert(np, class1, class2):
    out=[]
    for i in range(len(np)):
        if np[i] == class1:
            #print("Detected : ",class1)
            out.append(int(1)) 
        else:
            #print("Detected : ",class2)
            out.append(int(-1))
    return out

# Function that converts the output frame according to 2 features
def convert2(np, class1, class2, class3):
    out = []
    for i in range(len(np)):
        if np[i] == class1:
            out.append(int(0)) 
            
        elif np[i] == class2:
            out.append(int(1))  
            
        elif np[i] == class3:
            out.append(int(2))  
        
        else:
            out.append(int(-1))  
    return out

# Function that reverses the convert function
def Rconvert(np, class1, class2):
    out=[]
    for i in range(len(np)):
        if np[i] == 1:
            out.append(class1) 
        else:
            out.append(class2)
    return out

# Function that returns 2 dataframes based on the name
def getframes(frame1, frame2):
    df1= dfSetosa
    df2= dfSetosa
    
    if frame1 == SetOptions[0]:
        df1 = dfSetosa
    elif frame1 == SetOptions[1]:
        df1 = dfVersicolor
    elif frame1 == SetOptions[2]:
        df1 = dfVirginica

    if frame2 == SetOptions[0]:
        df2 = dfSetosa
    elif frame2 == SetOptions[1]:
        df2 = dfVersicolor
    elif frame2 == SetOptions[2]:
        df2 = dfVirginica
             
    return df1,df2

# Model 1 (*Task 1*)

In [6]:
# Training function
def model1train(dftrain ,CLASS1, CLASS2,feature1,feature2 ,EPOCHS, Lrate, BIAS, SEED):

    np.random.seed(SEED) # Set seed for random values
    weights = np.random.random(2) # Generate weights
    mean_error = 0.0
    bias = np.random.random()*BIAS # Generate bias
    c=0 # generic counter
    w1=[]
    w2=[]
    b=[]
    
    for i in range(EPOCHS):
        
        # Re-shuffle the train dataframe each epoch
        df_train = dftrain.sample(frac=1)
        
        # Get the train and features frames
        X_train, y_train = X_ysplit(df_train,feature1,feature2)
        
        # Convert them into numpy sets
        X_Train = X_train.to_numpy()
        y_Train = y_train.to_numpy()
        
        # Convert the output
        y_Train = convert(y_Train, CLASS1, CLASS2)
        
        # Change the seed value
        SEED += i
        
        for n in range(len(X_train)):
            
            w1.append(weights[0])
            w2.append(weights[1])
            b.append(bias)
            y_predicted = signum(weights.dot(X_Train[n].reshape(2,1)) + (bias*BIAS) ) 
            loss =  y_Train[n] - y_predicted
            weights = weights + (Lrate*loss*X_Train[n])
            bias += Lrate*loss*BIAS

            if y_predicted != y_Train[n]:
                mean_error += loss
        
        step( (float(i)/float(EPOCHS))*100 ) # Comment this line if you are not running the gui
        
        mean_error = mean_error/60   #Divide total error for each epoch by number of samples
    step(0) # Comment this line if you are not running the gui
    return weights, bias, mean_error,w1, w2, b

# Test function
def model1test(dftest, CLASS1, CLASS2,feature1,feature2 ,weights , bias, SEED):
    
    df_test = dftest.sample(frac=1)
    X_test, y_test = X_ysplit(df_test,feature1,feature2)
    
    X_test = X_test.to_numpy()
    y_test = y_test.to_numpy()
    y_test = convert(y_test, CLASS1, CLASS2)
    y_pred = []

    for i in range(len(X_test)):
        y_pred.append( signum(  ( (weights[0]*X_test[i,0]) + (weights[1]*X_test[i,1]) + bias)  ) ) 
        #print("Predicted : %d     Actual : %d"%(y_pred[i],y_test[i]))
    
    
    return y_test , y_pred

# Model 2 (*Task 2*)

In [7]:
def model2train(dftrain ,CLASS1, CLASS2,feature1,feature2 ,EPOCHS, Lrate, BIAS, SEED, THRES):

    np.random.seed(SEED) # Set seed for random values
    weights = np.random.random(2) # Generate weights
    squared_error = 0.0        #NEW VARIABLE -->The sum of squares of the loss each iteration
    mean_squared_error = 0.0   #NEW VARIABLE -->The squared error divided by the number of samples in an epoch to get MSE 
    bias = np.random.random()*BIAS # Generate bias
    c=0 # generic counter
    w1=[]
    w2=[]
    b=[]

    
    for i in range(EPOCHS):
        squared_error = 0.0
        # Re-shuffle the train dataframe each epoch
        df_train = dftrain.sample(frac=1)
        
        # Get the train and features frames
        X_train, y_train = X_ysplit(df_train,feature1,feature2)
        
        # Convert them into numpy sets
        X_Train = X_train.to_numpy()
        y_Train = y_train.to_numpy()
        
        # Convert the output
        y_Train = convert(y_Train, CLASS1, CLASS2)
        
        # Change the seed value
        SEED += i
        
        for n in range(len(X_train)):
            
            w1.append(weights[0])
            w2.append(weights[1])
            b.append(bias)
            
            y_predicted = weights.dot(X_Train[n].reshape(2,1)) + bias*BIAS
            loss =  y_Train[n] - y_predicted
            weights = weights + (Lrate*loss*X_Train[n])
            bias += Lrate*loss*BIAS
             
            if signum(y_predicted) != y_Train[n]:
                squared_error += (loss ** 2)
            
            
        
        step( (float(i)/float(EPOCHS))*100 ) # Comment this line if you are not running the gui
        
        mean_squared_error = squared_error/60  #Divide total of loss squares for each epoch by number of samples to get MSE
        
        if mean_squared_error <= THRES:  #If MSE is less than the given threshold value, THRES, stop training process
            break
        
    step(0) # Comment this line if you are not running the gui
    return weights, bias, mean_squared_error, w1, w2, b


def model2test(dftest, CLASS1, CLASS2,feature1,feature2 ,weights , bias, SEED):

    df_test = dftest.sample(frac=1)
    X_test, y_test = X_ysplit(df_test,feature1,feature2)
    
    X_test = X_test.to_numpy()
    y_test = y_test.to_numpy()
    y_test = convert(y_test, CLASS1, CLASS2)
    y_pred = []
    for i in range(len(X_test)):
        y_pred.append( signum(  ( (weights[0]*X_test[i,0]) + (weights[1]*X_test[i,1]) + bias)  ) ) 
        #print("Predicted : %d     Actual : %d"%(y_pred[i],y_test[i]))
    
    return y_test , y_pred

# Model 3 (*Task 3*)

In [8]:
# Train function
def model3train(X, y, hlayers, nneurons, lrate, epochs, afunc, seed):
     
    model = MLPClassifier( hidden_layer_sizes = (hlayers*[nneurons])
                          ,learning_rate ='constant'
                          ,learning_rate_init = lrate
                          ,max_iter = epochs
                          ,solver ='lbfgs'
                          ,activation = afunc
                         ,random_state = seed)
    model.fit(X, y)    
    return model

# Function that draws a neural network

In [9]:
def draw_neural_net(ax, left, right, bottom, top, layer_sizes):
    '''
    Draw a neural network cartoon using matplotilb.
    
    :usage:
        >>> fig = plt.figure(figsize=(12, 12))
        >>> draw_neural_net(fig.gca(), .1, .9, .1, .9, [4, 7, 2])
    
    :parameters:
        - ax : matplotlib.axes.AxesSubplot
            The axes on which to plot the cartoon (get e.g. by plt.gca())
        - left : float
            The center of the leftmost node(s) will be placed here
        - right : float
            The center of the rightmost node(s) will be placed here
        - bottom : float
            The center of the bottommost node(s) will be placed here
        - top : float
            The center of the topmost node(s) will be placed here
        - layer_sizes : list of int
            List of layer sizes, including input and output dimensionality
    '''
    n_layers = len(layer_sizes)
    v_spacing = (top - bottom)/float(max(layer_sizes))
    h_spacing = (right - left)/float(len(layer_sizes) - 1)
    # Nodes
    for n, layer_size in enumerate(layer_sizes):
        layer_top = v_spacing*(layer_size - 1)/2. + (top + bottom)/2.
        for m in range(layer_size):
            circle = plt.Circle((n*h_spacing + left, layer_top - m*v_spacing), v_spacing/4.,
                                color='w', ec='k', zorder=4)
            ax.add_artist(circle)
    # Edges
    for n, (layer_size_a, layer_size_b) in enumerate(zip(layer_sizes[:-1], layer_sizes[1:])):
        layer_top_a = v_spacing*(layer_size_a - 1)/2. + (top + bottom)/2.
        layer_top_b = v_spacing*(layer_size_b - 1)/2. + (top + bottom)/2.
        for m in range(layer_size_a):
            for o in range(layer_size_b):
                line = plt.Line2D([n*h_spacing + left, (n + 1)*h_spacing + left],
                                  [layer_top_a - m*v_spacing, layer_top_b - o*v_spacing], c='k')
                ax.add_artist(line)



# Tkinter GUI

In [ ]:
%%capture
# Global variables used
global dftest,w,b,m,w1_arr,w2_arr,b_arr
global model3, y_test, X_test

# Create main window
root = Tk()
root.title("CI Project")

# Adjust size and color
root.geometry( "1000x600" )
root['background']=palette[0]
    
####################################### Callback Functions #######################################
# Shows graph of 1 dataset
def show1():
    df2, df1=getframes(set1.get(),"")
    
    figure2 = plt.Figure(figsize=(plotLength,plotWidth), dpi=130)
    ax2 = figure2.add_subplot(111)
    ax2.scatter(df2[str(feature1.get())],df2[str(feature2.get())], color = 'r', label = str(set1.get()))
    scatter2 = FigureCanvasTkAgg(figure2, root) 
    scatter2.get_tk_widget().grid(column = 1, row = 1)
    ax2.set_xlabel((str(feature1.get())))
    ax2.set_ylabel(str(feature2.get()))
    ax2.set_title(str(set1.get()))
    ax2.legend(loc='upper left')
    root.lift()

# Shows graph of 2 datasets
def show2():
    
    df1, df2 = getframes(set1.get(),set2.get())
    figure1 = plt.Figure(figsize=(plotLength,plotWidth), dpi=130)
    ax1 = figure1.add_subplot(111)
    ax1.scatter(df1[str(feature1.get())],df1[str(feature2.get())], color = 'r', marker="o", label = str(set1.get()))
    ax1.scatter(df2[str(feature1.get())],df2[str(feature2.get())], color = 'g',marker="o", label = str(set2.get()))
    scatter1 = FigureCanvasTkAgg(figure1, root) 
    scatter1.get_tk_widget().grid(column = 1, row = 1)
    ax1.set_xlabel((str(feature1.get())))
    ax1.set_ylabel(str(feature2.get()))
    ax1.set_title(str(set1.get()) + " and " + str(set2.get()))
    ax1.legend(loc='upper left')
    root.lift()

# Shows graph of 3 datasets
def show3():
    df1 = dfSetosa
    df2 = dfVersicolor
    df3 = dfVirginica
    figure1 = plt.Figure(figsize=(plotLength,plotWidth), dpi=130)
    ax1 = figure1.add_subplot(111)
    ax1.scatter(df1[str(feature1.get())],df1[str(feature2.get())], color = 'r', marker="o", label = SetOptions[0])
    ax1.scatter(df2[str(feature1.get())],df2[str(feature2.get())], color = 'g',marker="o", label = SetOptions[1])
    ax1.scatter(df3[str(feature1.get())],df3[str(feature2.get())], color = 'b',marker="o", label = SetOptions[2])
    scatter1 = FigureCanvasTkAgg(figure1, root) 
    scatter1.get_tk_widget().grid(column = 1, row = 1)
    ax1.set_xlabel((str(feature1.get())))
    ax1.set_ylabel(str(feature2.get()))
    ax1.set_title("Setosa and Versicolor and Virginica")
    ax1.legend(loc='upper left')
    root.lift()

# Functions that update the dropdown lists
def changeValueofS1(text):
    options1 = [x for x in SetOptions if x != text.widget.get()]
    set1["values"] = options1
    button2.config(text = "Plot " + str(set1.get()) + " vs. " + str(set2.get()) )
    
def changeValueofS2(text):
    options2 = [x for x in SetOptions if x != text.widget.get()]
    set2["values"] = options2
    button1.config(text = "Plot " + str(set1.get()) + " only")
    button2.config(text = "Plot " + str(set1.get()) + " vs. " + str(set2.get()) )

def changeValueofF1(text):
    options3 = [x for x in FeatureOptions if x != text.widget.get()]
    feature1["values"] = options3

def changeValueofF2(text):
    options4 = [x for x in FeatureOptions if x != text.widget.get()]
    feature2["values"] = options4

def changeValueofS11(text):
    options1 = [x for x in SetOptions if x != text.widget.get()]
    set1["values"] = options1
    
def changeValueofS21(text):
    options2 = [x for x in SetOptions if x != text.widget.get()]
    set2["values"] = options2

def changeValueofF11(text):
    options3 = [x for x in FeatureOptions if x != text.widget.get()]
    feature1["values"] = options3

def changeValueofF21(text):
    options4 = [x for x in FeatureOptions if x != text.widget.get()]
    feature2["values"] = options4

# Function that steps the progress bar
def step(step=0):
    root.update_idletasks()
    pBar['value'] = step

# Function that enables the threshold field  
def editThreshold():
    if clicked12.get():
        exitField["state"] = "normal"
        
    else:
        exitField["state"] = "disabled"
        
        
# Recoloring the GUI
def recolor(text):
    try:
        palette[2] = themeDict[text.widget.get()]
    except:
        palette[2] = themeDict[themeOptions[0]]
  
    #Setting Labels
    dummyLabel1.configure(background = palette[0], fg = palette[2])
    set1Label.configure(background= palette[0], fg = palette[2])
    set2Label.configure(background= palette[0], fg = palette[2])
    feature1Label.configure(background= palette[0], fg = palette[2])
    feature2Label.configure(background= palette[0], fg = palette[2])
    biasLabel.configure(background=palette[0], fg = palette[2])
    epochsLabel.configure(background=palette[0], fg = palette[2])
    lrLabel.configure(background=palette[0], fg = palette[2])
    seedLabel.configure(background=palette[0], fg = palette[2])
    seed2Label.configure(background=palette[0], fg = palette[2])
    hlLabel.configure(background=palette[0], fg = palette[2])
    nnLabel.configure(background=palette[0], fg = palette[2])
    lr2Label.configure(background=palette[0], fg = palette[2])
    epochs2Label.configure(background=palette[0], fg = palette[2])
    bias2Label.configure(background=palette[0], fg = palette[2])
    afuncLabel.configure(background=palette[0], fg = palette[2])
    saturationLabel.configure(background=palette[0], fg = palette[2])
    
    #Setting Titles
    title1.configure(background= palette[1], fg = palette[2])
    title2.configure(background= palette[1], fg = palette[2])
    title3.configure(background= palette[1], fg = palette[2])
    title4.configure(background= palette[1], fg = palette[2])
    
    #Setting Buttons
    button1.configure(relief="flat", background=palette[2])
    button2.configure(relief="flat", background=palette[2])
    button3.configure(relief="flat", background=palette[2])
    trainbutton.configure(relief="flat", background=palette[2])
    testbutton.configure(relief="flat", background=palette[2])
    weightsbutton.configure(relief="flat", background=palette[2])
    biasbutton.configure(relief="flat", background=palette[2])
    train2button.configure(relief="flat", background=palette[2])
    test2button.configure(relief="flat", background=palette[2])
    creditsbutton.configure(relief="flat", background=palette[2])
    
    #Setting Fields and boxes
    biasBox.configure(background=palette[2])
    bias2Box.configure(background=palette[2])
    lrField.configure(background=palette[2])
    epochsField.configure(background=palette[2])
    seedField.configure(background=palette[2])
    seed2Field.configure(background=palette[2])
    hlField.configure(background=palette[2])
    nnField.configure(background=palette[2])
    lr2Field.configure(background=palette[2])
    epochs2Field.configure(background=palette[2])
    saturationBox.configure(background=palette[2])
    exitField.configure(background=palette[2])
   
    #Setting Progress Bar
    style=ttk.Style()
    style.theme_use("alt")
    style.configure("Custom_style.Horizontal.TProgressbar",foreground=palette[0],troughcolor=palette[0],
                    background=palette[2], bordercolor=palette[2],lightcolor=palette[2],darkcolor=palette[2])
    pBar.configure(style= "Custom_style.Horizontal.TProgressbar")

# Function that plots the weights variation
def plotweights():
    global w1_arr, w2_arr
    
    figure1 = plt.Figure(figsize=(plotLength,plotWidth), dpi=130)
    ax2 = figure1.add_subplot(111)
    line1 = FigureCanvasTkAgg(figure1, root)
    line1.get_tk_widget().grid(column = 1, row = 1)
    y1= np.arange(0, max(len(w1_arr),len(w2_arr)),1)
    ax2.plot( y1, w1_arr, color="red", label ="weight 1")
    ax2.plot( y1, w2_arr, color="green", label = "weight 2")
    ax2.set_xlabel("Iteration")
    ax2.set_ylabel("Weights value")
    ax2.set_title("Weights variation")
    ax2.legend(loc='upper left')
    root.lift()

# Function that plots the bias variation
def plotbiases():
    global b_arr
    
    figure1 = plt.Figure(figsize=(plotLength,plotWidth), dpi=130)
    ax2 = figure1.add_subplot(111)
    line1 = FigureCanvasTkAgg(figure1, root)
    line1.get_tk_widget().grid(column = 1, row = 1)
    y1= np.arange(0, len(b_arr),1)
    ax2.plot( y1, b_arr, color="red")
    ax2.set_xlabel("Iteration")
    ax2.set_ylabel("Bias")
    ax2.set_title("Bias variation")
    root.lift()

# Function that runs model 1 and 2
def runmodel12():
    global dftest,w,b,m,w1_arr, w2_arr ,b_arr
    df1, df2 = getframes(set1.get(),set2.get())
    dftrain, dftest = train_testsplit(df1,df2,1)
    
    if clicked12.get():
        
        w, b, m, w1_arr, w2_arr, b_arr = model2train(dftrain
                                                     , set1.get(), set2.get()
                                                     , feature1.get() , feature2.get() ,
                         int(epochsField.get()), float(lrField.get()) , clicked10.get(), int(seedField.get())
                                                    , float(exitField.get()))
        
    
    else:
        
        w, b, m, w1_arr, w2_arr ,b_arr = model1train(dftrain
                                                     , set1.get(), set2.get()
                                                     , feature1.get(), feature2.get() ,
                         int(epochsField.get()), float(lrField.get()) , clicked10.get(), int(seedField.get()))
        
    
    
    figure1 = plt.Figure(figsize=(plotLength,plotWidth), dpi=130)
    ax1 = figure1.add_subplot(111)
    ax1.scatter(df1[str(feature1.get())],df1[str(feature2.get())], color = 'r', marker="o", label = str(set1.get()))
    ax1.scatter(df2[str(feature1.get())],df2[str(feature2.get())], color = 'g',marker="o", label = str(set2.get()))
    scatter1 = FigureCanvasTkAgg(figure1, root)
    scatter1.get_tk_widget().grid(column = 1, row = 1)
    ax1.set_xlabel((str(feature1.get())))
    ax1.set_ylabel(str(feature2.get()))
    ax1.set_title(str(set1.get()) + " and " + str(set2.get()) + " with Bias = %f "%(b) )
    ax1.legend(loc='upper left')
    root.lift()
    
    max1 = df1[str(feature1.get())].max()
    min1 = df1[str(feature1.get())].min()
    max2 = df2[str(feature1.get())].max()
    min2 = df2[str(feature1.get())].min()
    
    x = np.arange(min(min1,min2), max(max1,max2), 0.01)
    if b==0:
        slope = -(w[0]/w[1])  
        
    else:
        slope = -(b/w[1])/(b/w[0])  
    
    intercept = -b/w[1]
    ax1.plot(x, slope*x + intercept)
    
    testbutton["state"]="normal"
    weightsbutton["state"]="normal"
    biasbutton["state"]="normal"
    
# Function that tests model 1   
def testmodel12():
    
    global dftest,w,b,m
    y_test, y_pred = model1test(dftest,set1.get(),set2.get(),feature1.get() 
                               ,feature2.get() , w, b, int(seedField.get()) )
    
    
    y_test = Rconvert(y_test, set1.get(), set2.get())
    y_pred = Rconvert(y_pred, set1.get(), set2.get())
    
    
    conf_matrix = sklearn.metrics.confusion_matrix(y_true=y_test, y_pred=y_pred
                                                   , labels=[set1.get(),set2.get()])
    
    fig = plt.Figure(figsize=(plotLength,plotWidth), dpi=130)
    ax = fig.add_subplot(111)
    ax.matshow(conf_matrix, cmap=plt.cm.Blues, alpha=0.3)
    alpha=[set1.get() , set2.get()]
    ax.set_xticklabels(['']+alpha)
    ax.set_yticklabels(['']+alpha)
    for i in range(conf_matrix.shape[0]):
        for j in range(conf_matrix.shape[1]):
            ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', ha='center', size='xx-large')

    ax.set_xlabel('Predictions', fontsize=14)
    ax.set_ylabel('Actuals', fontsize=14)
    ax.set_title('Confusion Matrix', fontsize=14)
    conf = FigureCanvasTkAgg(fig, root)

    conf.get_tk_widget().grid(column = 1, row = 1)
    root.lift()

# Function that runs model 3
def runmodel3():
    global model3, X_test, y_test
    
    df = pd.concat([dfSetosa, dfVersicolor, dfVirginica], ignore_index=True)
    df = df.sample(frac=1)
    y = df.iloc[:,-1]
    X = df.iloc[:, : 4]
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, train_size=RATIO, random_state=1)
    
    model3 = model3train( X_train
                         ,y_train
                         ,int(hlField.get())  #hidden layers
                        ,int(nnField.get())  #neuron per layer
                        ,float(lr2Field.get()) #learning rate
                        ,int(epochs2Field.get()) #epochs
                        ,str(afunc.get()) #activation func
                        ,int(seed2Field.get()) #seed
                       )
    
    test2button["state"]="normal"
    inlayer = [4]
    hidden = int(hlField.get())*[int(nnField.get())]
    outlayer = [3]
     
    
    fig = plt.figure(figsize=(plotLength*2,plotWidth*2), dpi=160)
    ax = fig.gca()
    ax.axis('off')
    draw_neural_net(ax, .1, .9, .9, .1, inlayer + hidden + outlayer)
    fig.savefig('nnmodel.png')
    
    
    img_arr = mpimg.imread('nnmodel.png')
    
    figure2 = plt.Figure(figsize=(plotLength,plotWidth), dpi=130)
    ax2 = figure2.add_subplot(111)
    ax2.imshow(img_arr, aspect = 'auto')
    ax2.axis('off')
    scatter2 = FigureCanvasTkAgg(figure2, root) 
    scatter2.get_tk_widget().grid(column = 1, row = 1)
    root.lift()
    
# Function that tests model 3
def testmodel3():
    global model3, X_test, y_test
    acc = model3.score(X_test, y_test)

    y_pred = model3.predict(X_test)

    cm = confusion_matrix(y_test, y_pred)
    
    
    conf_matrix = sklearn.metrics.confusion_matrix(y_true=y_test, y_pred=y_pred
                                                   , labels=[SetOptions[0], SetOptions[1], SetOptions[2]])
    
    fig = plt.Figure(figsize=(plotLength,plotWidth), dpi=130)
    ax = fig.add_subplot(111)
    ax.matshow(conf_matrix, cmap=plt.cm.Blues, alpha=0.3)
    alpha=[SetOptions[0], SetOptions[1], SetOptions[2]]
    ax.set_xticklabels(['']+alpha)
    ax.set_yticklabels(['']+alpha)
    for i in range(conf_matrix.shape[0]):
        for j in range(conf_matrix.shape[1]):
            ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', ha='center', size='xx-large')

    ax.set_xlabel('Predictions', fontsize=14)
    ax.set_ylabel('Actuals', fontsize=14)
    ax.set_title('Confusion Matrix with accuracy %f'%(acc*100), fontsize=14)
    conf = FigureCanvasTkAgg(fig, root)
    conf.get_tk_widget().grid(column = 1, row = 1)
    root.lift()
    
def showcredits():
    img_arr = mpimg.imread('cover.png')
    
    figure2 = plt.Figure(figsize=(plotLength,plotWidth), dpi=160)
    ax2 = figure2.add_subplot(111)
    ax2.imshow(img_arr,aspect='auto')
    ax2.axis('off')
    scatter2 = FigureCanvasTkAgg(figure2, root) 
    scatter2.get_tk_widget().grid(column = 1, row = 1)
    root.lift()
    
####################################### Declaring GUI Variables #######################################
# Datatype of menu text
clicked1 = StringVar()
clicked2 = StringVar()
clicked3 = StringVar()
clicked4 = StringVar()
clicked5 = StringVar()
clicked6 = StringVar()
clicked7 = StringVar()
clicked8 = StringVar()
clicked9 = StringVar()
clicked10 = IntVar()
clicked11 = IntVar()
clicked12 = IntVar()

# Create Combo boxes
set1 = ttk.Combobox(root , width = 13, textvariable = clicked1, state="readonly", font=("Arial", 11))
set2 = ttk.Combobox(root , width = 13, textvariable = clicked2, state="readonly", font=("Arial", 11))

feature1 = ttk.Combobox(root , width = 13, textvariable = clicked3, state="readonly", font=("Arial", 11))
feature2 = ttk.Combobox(root , width = 13, textvariable = clicked4, state="readonly", font=("Arial", 11))


afunc = ttk.Combobox(root , width = 8, textvariable = clicked7, state="readonly", font=("Arial", 11))

theme = ttk.Combobox(root , width = 8, textvariable = clicked9, state="readonly", font=("Arial", 11))

# Binding combo boxes to callback functions
set1.bind("<<ComboboxSelected>>", changeValueofS2)
set2.bind("<<ComboboxSelected>>", changeValueofS1)

feature1.bind("<<ComboboxSelected>>", changeValueofF2)
feature2.bind("<<ComboboxSelected>>", changeValueofF1)

theme.bind("<<ComboboxSelected>>", recolor)


# Initial menu text
set1["values"] = SetOptions
set1.current(0)
set2["values"] = SetOptions
set2.current(1)

feature1["values"] = FeatureOptions
feature1.current(0)
feature2["values"] = FeatureOptions
feature2.current(1)

theme["values"] = themeOptions
theme.current(0)


afunc["values"] = ['logistic', 'tanh']
afunc.current(0)

# Create buttons
button1 = tk.Button( root , text = "Plot " + str(set1.get()) + " only" , command = show1 ,width = 20
                    ,font=("Arial", 9,'bold'))
button2 = tk.Button( root , text = "Plot " + str(set1.get()) + " vs. " + str(set2.get()) , command = show2,width = 30  
                 ,font=("Arial", 9,'bold'))
button3 = tk.Button( root , text = "Plot all" , command = show3,width = 15
                 ,font=("Arial", 9,'bold') )
trainbutton = tk.Button (root , text = "Train Model" , command = runmodel12 ,width = 15, 
                 font=("Arial", 9,'bold'))
testbutton = tk.Button (root , text = "Test Model" , command = testmodel12 ,width = 15, 
                 font=("Arial", 9,'bold'), state="disabled")
weightsbutton = tk.Button (root , text = "Plot weights" , command = plotweights ,width = 15, 
                 font=("Arial", 9,'bold'), state="disabled")

biasbutton = tk.Button (root , text = "Plot Bias" , command = plotbiases ,width = 15, 
                 font=("Arial", 9,'bold'), state="disabled")
train2button = tk.Button (root , text = "Train Model" , command = runmodel3 ,width = 15, 
                 font=("Arial", 9,'bold'))
test2button = tk.Button (root , text = "Test Model" , command = testmodel3 ,width = 15, 
                 font=("Arial", 9,'bold'), state="disabled")


creditsbutton = tk.Button(root, text = 'Credits',  width = 15,command = showcredits, font=("Arial", 9,'bold'))

# Create check boxes
biasBox =  tk.Checkbutton (root, variable=clicked10, onvalue=1, offvalue=0)
bias2Box =  tk.Checkbutton (root, variable=clicked11, onvalue=1, offvalue=0)
saturationBox =  tk.Checkbutton (root, variable=clicked12, onvalue=1, offvalue=0, command= editThreshold)

# Create text fields
lrField = tk.Entry(root, width = 8, font=("Arial", 9,'bold'))
lrField.insert(0,"0.01")

epochsField = tk.Entry(root, width = 8, font=("Arial", 9,'bold'))
epochsField.insert(0,"80")

seedField = tk.Entry(root, width = 8, font=("Arial", 9,'bold'))
seedField.insert(0,"2")
seed2Field = tk.Entry(root, width = 8, font=("Arial", 9,'bold'))
seed2Field.insert(0,"2")

hlField= tk.Entry(root, width = 8, font=("Arial", 9,'bold'))
hlField.insert(0,"3")

nnField= tk.Entry(root, width = 8, font=("Arial", 9,'bold'))
nnField.insert(0,"3")

lr2Field = tk.Entry(root, width = 8, font=("Arial", 9,'bold'))
lr2Field.insert(0,"0.01")

epochs2Field = tk.Entry(root, width = 8, font=("Arial", 9,'bold'))
epochs2Field.insert(0,"80")

exitField = tk.Entry(root, width = 11, font=("Arial", 9,'bold'))
exitField.insert(0,"0.1")
exitField["state"]= "disabled"

# Create Labels
dummyLabel1 = tk.Label( root , text = "Accent :  ",font=("Arial", 10,'bold') )

label = tk.Label( root , text = "       " )
label.configure(background= palette[0])

title1 = tk.Label( root , text = "Single Layer \n Perceptron Model  " , font=("Arial", 15,'bold'))
title2 = tk.Label( root , text = "  Analysis  ", font=("Arial", 15,'bold'))
title3 = tk.Label( root , text = " Multi-Layer \n Perceptron Model  " , font=("Arial", 15,'bold'))
title4 = tk.Label( root , text = " Settings " , font=("Arial", 15,'bold'))

set1Label = tk.Label( root , text = "Set 1" , font=("Arial", 11,'bold'))
set2Label = tk.Label( root , text = "Set 2" , font=("Arial", 11,'bold'))

feature1Label = tk.Label( root , text = "Feature 1" , font=("Arial", 11,'bold'))
feature2Label = tk.Label( root , text = "Feature 2" , font=("Arial", 11,'bold'))

lrLabel = tk.Label( root, text = "Learning rate : ", font=("Arial", 11,'bold'))
epochsLabel = tk.Label( root, text = "Number of Epochs :", font=("Arial", 11,'bold'))
biasLabel = tk.Label( root, text = "Use Bias ? ",justify="right",  font=("Arial", 11,'bold'))
seedLabel = tk.Label( root, text = "Seed : ",justify="right",  font=("Arial", 11,'bold'))

hlLabel= tk.Label( root, text = "Hidden layers : ", font=("Arial", 11,'bold'))
nnLabel = tk.Label( root, text = "Neurons per layer : ", font=("Arial", 11,'bold'))
lr2Label = tk.Label( root, text = "Learning rate : ", font=("Arial", 11,'bold'))
epochs2Label = tk.Label( root, text = "Number of epochs : ", font=("Arial", 11,'bold'))
bias2Label = tk.Label( root, text = "Use bias ?", font=("Arial", 11,'bold'))
afuncLabel = tk.Label( root, text = "Activation function :", font=("Arial", 11,'bold'))
seed2Label = tk.Label( root, text = "Seed :", font=("Arial", 11,'bold'))

saturationLabel = tk.Label( root, text = "Auto exit ?", font=("Arial", 11,'bold'))

# Create Progress bar
pBar = Progressbar(root, orient=HORIZONTAL, length=200, mode='determinate')

####################################### Grid the GUI elements #######################################

title4.grid(column = 7, row = 1, padx = 10)
label.grid(column = 5, row=1)
title2.grid(column = 3, row = 1, pady = 10, padx= 20)

set1Label.grid(column = 2, row = 2, pady= 5)
set2Label.grid(column = 4, row = 2, pady= 5)
dummyLabel1.grid(column = 6, row = 2, padx = 10)
theme.grid(column = 7, row = 2, pady= 5, padx=10)

creditsbutton.grid(column = 7, row=3)
set1.grid(column = 2, row = 3, pady= 5)
set2.grid(column = 4, row = 3, pady= 5)
                 
feature1Label.grid(column = 2, row = 4, pady= 5)
feature2Label.grid(column = 4, row = 4, pady= 5)

feature1.grid(column = 2, row = 5, pady= 5)
feature2.grid(column = 4, row = 5, pady= 5)

button1.grid(column=2, row=6, padx = 20 , pady = 15)
button2.grid(column=3, row=6, padx = 20)
button3.grid(column=4, row=6, padx = 20)

title1.grid(column = 3, row = 7, pady = 10, padx= 20)
title3.grid(column = 7, row = 7, pady = 10, padx= 20)

lrLabel.grid(column = 2, row = 14, pady= 5)
lrField.grid(column = 3, row = 14, pady= 5)
hlLabel.grid(column = 6, row = 14, pady= 5)
hlField.grid(column = 7, row = 14, pady= 5)

epochsLabel.grid(column = 2, row = 15, pady= 5)
epochsField.grid(column = 3, row = 15, pady= 5)
nnLabel.grid(column = 6, row = 15, pady= 5)
nnField.grid(column = 7, row = 15, pady= 5)

biasLabel.grid(column = 2, row = 16, pady= 5)
biasBox.grid(column = 3, row = 16, pady= 5)
lr2Label.grid(column = 6, row = 16, pady= 5)
lr2Field.grid(column = 7, row = 16, pady= 5)

seedLabel.grid(column = 2, row = 17, pady= 5)
seedField.grid(column = 3, row = 17 , pady = 5)
epochs2Label.grid(column = 6, row = 17, pady= 5)
epochs2Field.grid(column = 7, row = 17, pady= 5)

saturationLabel.grid(column = 2, row = 18 , pady = 5)
saturationBox.grid(column = 3, row = 18 , pady = 5)
exitField.grid(column = 4, row = 18 , pady = 5)
afuncLabel.grid(column = 6, row = 18 , pady = 5)
afunc.grid(column = 7, row = 18 , pady = 5)

trainbutton.grid(column = 2, row = 19 , pady = 15)
pBar.grid(column = 3, row = 19 , pady = 15)
testbutton.grid(column = 4, row=19)
seed2Label.grid(column = 6, row = 19 , pady = 5)
seed2Field.grid(column = 7, row =19 , pady = 5)

weightsbutton.grid(column = 2, row=20)
biasbutton.grid(column = 3, row=20)
train2button.grid(column = 6, row = 20, pady = 5)
test2button.grid(column = 7, row = 20, pady = 5)

####################################### Run the GUI #######################################
recolor(clicked9)
# Create graphs window
root= tk.Tk()
root.title("Graphs")
# Execute tkinter
root.mainloop()